# Scipy / matplotlib tutorial

Please copy your solution of the prisoner riddle here:

How many turns are required for a population of 50 prisoners? Repeat this simulation 1000 times and plot a histogram to visualize the uncertainty.

Plot the number of required turns against the prisoner population size (ranging from 50 to 200).

What kind of relationship do we see? Can you parameterize this, and fit a function?

Plot the same, but now extrapolate untill a prisoner population size of 400.

# Pandas tutorial

We will work with some simulated data from a typical 2-alternative forced choice reaction time task. Subjects see two Gabors on the left and right side of the screen and have to judge which Gabor is of higher contrast. There is an easy and hard condition, which is manipulated by the signal strength: the contrast difference between the two Gabors. It is a pharmacolical study, and subjects partipate once after taking a drug, and once after taking a placebo pill.

Every row in the dataframe corresponds to one trial of the experiment. For every trial, we store the following information across the columns:
- 'subj_idx' codes for the subject identity (names)
- 'stimulus' codes for the location of higher contrast Gabor (0=left; 1=right).
- 'response' codes for the subject's choice (0=left; 1=right)
- 'rt' codes for the subjects reaction time (in seconds)
- 'signal_strength' codes for signal strength ('weak' or 'strong')
- 'drug' codes for drug condition ('placebo' or 'drug')

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy import stats
import matplotlib.pyplot as plt

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


Let's first load the data:

In [ ]:
df = pd.read_csv('data.csv')

Let's start with inspecting the dataframe. Print the first five row of the dataframe:

Print the number of rows in the dataframe:


Print all columns in the dataframe:

Print all unique subjects in the dataframe:

Print all unique stimuli in the dataframe:

Print all unique responses in the dataframe:

Print all unique difficulties in the dataframe:

Print all unique drug conditions in the dataframe:

Make a histogram of all reaction times in the dataframe:

Index the dataframe by the strong signals:

Decisions based on strong signals are typically made a little faster than based on weak signals (in this data that is not really the case however...). Make separate histograms for the reaction times of low and high signal strength trials:

Index the dataframe by the first subject:

Add a column that codes for whether the subject made correct response on each trial:

With a for-loop, compute the fraction of correct trials for each subject. Add those to an empty list.

With a nested for-loop, compute the fraction of correct trials for each subject, each drug condition and each signal strength condition:

Which value belongs to which subject, drug condition and signal strength? Note how this is going to require a lot of bookkeeping....

With a groupby statement, compute the fraction of correct trials for each subject:

With a groupby statement, compute the fraction of correct trials for each subject, each drug condition and each signal strength condition. Put the result in a new variable called 'res' and print this.

The result is a dataframe that contains a MultiIndex. This is also known as a hierarchical index dataframe. This allows you to have multiple columns acting as a row identifier and multiple rows acting as a header identifier. With MultiIndex, you can do some sophisticated data analysis, especially for working with higher dimensional data. Accessing data is the first step when working on a MultiIndex DataFrame.

the .loc method now takes values, one for each MultiIndex. For example, we can get the accuracy scores of subject 0, in the placebo condition, and for each signal strength, with: 

```
res.loc[0, 'placebo', :]
```

Index 'res' and obtain the accuracy scores of subjects for the strong signal strength trials in the placebo sessions:

Do the same for the low signal strength trials in placebo sessions:

There seems to be a difference, but is it statistically significant? Use a paired samples t-test.

Suppose we want to test whether there is an effect of drug condition on accuracy, but irrespective of signal strength. Now, we need to 'collapse' across signal strength; in other words, for every subject and drug condition, we need to take the mean across the two signal strengts. Do this with a two groupby statements (chained).

Check whether there is a significant difference of drug on accuracy:

In the Results section of your manuscript, you might want to report the grand average accury for each drug and signal strength condition. So, first you want to compute the accuracy for each subject, drug and signal strength condition, and then take the average across subjects, within each of the four cells of the design. Do this in one line: 

As rigorous scientists, we also want to report the variability across subjects. So, do the same, but now take the standard error of the mean across subjects:

Rather than taking the mean, we can apply any arbitrary function. For example,we might be interested to compute signal detection theoretic measures d' and criterion (https://en.wikipedia.org/wiki/Detection_theory). 

First, compute the total number of hits, misses, false alarms, and correct
rejects.
- Hit: when the stimulus was 1 and the subject (correctly) responded with 1
- Miss: when the stimulus was 1 and the subject (incorrectly) responded with 0
- False alarm: when the stimulus was 0 and the subject (incorrectly) responded with 1
- Correct reject: when the stimulus was 0 and the subject (correctly) responded with 0

Then, compute the hit-rate and false alarm-rates:
- hit_rate: fraction of hits on all stimulus==1 trials
- fa_rate: fraction of false alarm on all stimulus==0 trials

In signal detection theory, these measures are z-scored in the following way:

In [ ]:
hit_rate_z = sp.stats.norm.isf(1-hit_rate)
fa_rate_z = sp.stats.norm.isf(1-fa_rate)

You can compute signal detection theoretic d' and criterion as follows:
- d': difference between z-scored hit and false alarm rates
- criterion: average of z-scored hit and false alarm rates, times -1

Now put everything in a function that takes a dataframe as input, and returns the d' and criterion

Apply this function to every subject, drug and signal strenth condition.

Use seaborn to make boxplots of d', separately for each signal strength and drug condition.

In [ ]:
import seaborn as sns


Use a repeated measures ANOVA to test for main effects of signal and drug.

In [ ]:
from statsmodels.stats.anova import AnovaRM


Do the same for criterion.

In a scatter plot, plot subject's choice bias (criterion) on placebo sessions versus the change in bias for drug compared to placebo. Print the correlation coefficient.

Let's add 95% confidence intervals. You will need to bootstrap (https://en.wikipedia.org/wiki/Bootstrapping_(statistics)) those. You can use 

```df.groupby(['subj_idx', 'drug']).sample(frac=1, replace=True)``` 

to resample your dataframe.

This across subject's correlation looks compelling! The conservative subjects (postive on x) get less conservative when taking the drug (negative on y), and the liberal subjects (negative on x) get less liberal when taking the drug (positive on y). Can we conclude that this drug reduces subject's choice bias, irrespective of it's sign?

We have to be very aware of regression towards the mean: the phenomenon where if one sample of a random variable is extreme, the next sampling of the same random variable is likely to be closer to its mean (https://en.wikipedia.org/wiki/Regression_toward_the_mean).

We can correct for this with a permutation test. You can do this by computing the same across-subject correlation 10K times, each time using randomly assigned “placebo” vs “drug” labels of each subject. Then one can comapre the observed correlation coefficient (reflecting the combined effects of regression to the mean and the true relationship) against this permutation distribution.

Implement this analysis.